In [2]:
# Core libraries
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

# XML parsing (for BMRS API responses)
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Data visualization
import plotly.express as px
import plotly.graph_objects as go

# Utilities
import time
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("All libraries imported successfully")


All libraries imported successfully


In [3]:
# API Configuration
BASE_URL = 'https://api.bmreports.com/BMRS'

# Date range for testing (last 24 hours)
end_date = datetime.now()
start_date = end_date - timedelta(hours=24)

print(f"Testing date range:")
print(f"  Start: {start_date}")
print(f"  End: {end_date}")

# Endpoints to test
endpoints_to_test = {
    'FUELHH': 'Half-hourly generation by fuel type',
    'FUELINST': 'Instantaneous generation',
    'B1620': 'Actual generation output',
    'B1630': 'Actual or estimated wind and solar'
}

print(f"\n📡 Will test {len(endpoints_to_test)} endpoints")

Testing date range:
  Start: 2026-01-15 12:09:56.620753
  End: 2026-01-16 12:09:56.620753

📡 Will test 4 endpoints


In [5]:
# Test FUELHH endpoint (Half-hourly generation)
endpoint = 'FUELHH'
url = f'{BASE_URL}/{endpoint}/v1'

params = {
    'FromDateTime': start_date.strftime('%Y-%m-%d %H:%M:%S'),
    'ToDateTime': end_date.strftime('%Y-%m-%d %H:%M:%S'),
    'ServiceType': 'xml'
}

print(f"Testing endpoint: {url}")
print(f"Parameters: {params}")
print("\n" + "="*60)

try:
    response = requests.get(url, params=params, timeout=30)
    print(f"Status Code: {response.status_code}")
    print(f"Content-Type: {response.headers.get('Content-Type', 'Unknown')}")
    print(f"Response length: {len(response.text):,} characters")
    
    if response.status_code == 200:
        print("\nFirst 1000 characters of response:")
        print(response.text[:1000])
    else:
        print(f"\n Error response:")
        print(response.text[:500])
        
except Exception as e:
    print(f" Exception occurred: {str(e)}")

Testing endpoint: https://api.bmreports.com/BMRS/FUELHH/v1
Parameters: {'FromDateTime': '2026-01-15 12:09:56', 'ToDateTime': '2026-01-16 12:09:56', 'ServiceType': 'xml'}

 Exception occurred: HTTPSConnectionPool(host='api.bmreports.com', port=443): Max retries exceeded with url: /BMRS/FUELHH/v1?FromDateTime=2026-01-15+12%3A09%3A56&ToDateTime=2026-01-16+12%3A09%3A56&ServiceType=xml (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000020DDADED850>, 'Connection to api.bmreports.com timed out. (connect timeout=30)'))


In [6]:
# Try the newer Elexon Portal API (more modern)
print("Trying Elexon Portal API...")

elexon_url = "https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type"

# Simpler parameters for Elexon
params = {
    'from': start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
    'to': end_date.strftime('%Y-%m-%dT%H:%M:%SZ')
}

try:
    response = requests.get(elexon_url, params=params, timeout=30)
    print(f"✅ Status Code: {response.status_code}")
    print(f"Response length: {len(response.text):,} characters")
    
    if response.status_code == 200:
        # This API returns JSON
        data = response.json()
        print(f"\n📊 Response structure:")
        print(json.dumps(data, indent=2)[:1000])
    else:
        print(f"\n❌ Error: {response.text[:500]}")
        
except Exception as e:
    print(f"❌ Exception: {str(e)}")

Trying Elexon Portal API...
✅ Status Code: 200
Response length: 40,608 characters

📊 Response structure:
{
  "metadata": {
    "datasets": [
      "AGPT"
    ]
  },
  "data": [
    {
      "startTime": "2026-01-15T12:30:00Z",
      "settlementPeriod": 26,
      "data": [
        {
          "businessType": "Production",
          "psrType": "Biomass",
          "quantity": 1387.0
        },
        {
          "businessType": "Production",
          "psrType": "Fossil Gas",
          "quantity": 17778.0
        },
        {
          "businessType": "Production",
          "psrType": "Fossil Hard coal",
          "quantity": 0.0
        },
        {
          "businessType": "Production",
          "psrType": "Fossil Oil",
          "quantity": 0.0
        },
        {
          "businessType": "Production",
          "psrType": "Hydro Pumped Storage",
          "quantity": 78.0
        },
        {
          "businessType": "Production",
          "psrType": "Hydro Run-of-river and po

In [7]:
# Parse the full response
response_json = response.json()

print(f"Total data points: {len(response_json['data'])}")
print(f"Dataset: {response_json['metadata']['datasets']}")

# Look at one complete settlement period
print("\n📋 Sample settlement period structure:")
print(json.dumps(response_json['data'][0], indent=2))

# Extract all unique fuel types
fuel_types = set()
for period in response_json['data']:
    for entry in period['data']:
        fuel_types.add(entry['psrType'])

print(f"\n⚡ Available fuel types ({len(fuel_types)}):")
for fuel in sorted(fuel_types):
    print(f"  - {fuel}")

Total data points: 45
Dataset: ['AGPT']

📋 Sample settlement period structure:
{
  "startTime": "2026-01-15T12:30:00Z",
  "settlementPeriod": 26,
  "data": [
    {
      "businessType": "Production",
      "psrType": "Biomass",
      "quantity": 1387.0
    },
    {
      "businessType": "Production",
      "psrType": "Fossil Gas",
      "quantity": 17778.0
    },
    {
      "businessType": "Production",
      "psrType": "Fossil Hard coal",
      "quantity": 0.0
    },
    {
      "businessType": "Production",
      "psrType": "Fossil Oil",
      "quantity": 0.0
    },
    {
      "businessType": "Production",
      "psrType": "Hydro Pumped Storage",
      "quantity": 78.0
    },
    {
      "businessType": "Production",
      "psrType": "Hydro Run-of-river and poundage",
      "quantity": 512.0
    },
    {
      "businessType": "Production",
      "psrType": "Nuclear",
      "quantity": 3919.0
    },
    {
      "businessType": "Production",
      "psrType": "Other",
      "quantity"

In [8]:
# Convert nested JSON to flat DataFrame
records = []

for period in response_json['data']:
    timestamp = period['startTime']
    settlement_period = period['settlementPeriod']
    
    for entry in period['data']:
        records.append({
            'timestamp': timestamp,
            'settlement_period': settlement_period,
            'fuel_type': entry['psrType'],
            'generation_mw': entry['quantity']
        })

df = pd.DataFrame(records)

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort by timestamp
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"✅ Created DataFrame with {len(df)} rows")
print(f"\nShape: {df.shape}")
print(f"\nFirst few rows:")
df.head(10)

✅ Created DataFrame with 495 rows

Shape: (495, 4)

First few rows:


,timestamp,settlement_period,fuel_type,generation_mw
0,2026-01-15 12:30:00+00:00,26,Biomass,1387.000
1,2026-01-15 12:30:00+00:00,26,Fossil Gas,17778.000
2,2026-01-15 12:30:00+00:00,26,Fossil Hard coal,0.000
3,2026-01-15 12:30:00+00:00,26,Fossil Oil,0.000
4,2026-01-15 12:30:00+00:00,26,Hydro Pumped Storage,78.000
5,2026-01-15 12:30:00+00:00,26,Hydro Run-of-river and poundage,512.000
6,2026-01-15 12:30:00+00:00,26,Nuclear,3919.000
7,2026-01-15 12:30:00+00:00,26,Other,517.000
8,2026-01-15 12:30:00+00:00,26,Solar,1257.000
9,2026-01-15 12:30:00+00:00,26,Wind Offshore,7778.758


In [9]:
# Basic statistics
print("📊 Data Summary:")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Total records: {len(df)}")
print(f"Unique timestamps: {df['timestamp'].nunique()}")
print(f"Fuel types: {df['fuel_type'].nunique()}")

print("\n💡 Generation by fuel type (last period):")
last_period = df[df['timestamp'] == df['timestamp'].max()]
last_period_summary = last_period[['fuel_type', 'generation_mw']].sort_values('generation_mw', ascending=False)
print(last_period_summary.to_string(index=False))

print(f"\n⚡ Total generation: {last_period['generation_mw'].sum():,.0f} MW")



📊 Data Summary:
Date range: 2026-01-15 12:30:00+00:00 to 2026-01-16 10:30:00+00:00
Total records: 495
Unique timestamps: 45
Fuel types: 11

💡 Generation by fuel type (last period):
                      fuel_type  generation_mw
                     Fossil Gas      19949.000
                  Wind Offshore       6004.507
                        Nuclear       3954.000
                          Solar       3696.000
                   Wind Onshore       2988.837
                        Biomass       2002.000
Hydro Run-of-river and poundage        552.000
                          Other        406.000
           Hydro Pumped Storage          2.000
               Fossil Hard coal          0.000
                     Fossil Oil          0.000

⚡ Total generation: 39,554 MW


In [10]:
# Calculate renewable vs fossil
renewables = ['Solar', 'Wind Offshore', 'Wind Onshore', 'Hydro Run-of-river and poundage', 'Hydro Pumped Storage']
renewable_mw = last_period[last_period['fuel_type'].isin(renewables)]['generation_mw'].sum()
total_mw = last_period['generation_mw'].sum()

print(f"🌱 Renewable percentage: {(renewable_mw/total_mw)*100:.1f}%")

🌱 Renewable percentage: 33.5%


In [11]:
# Test fetching a longer period (last 7 days)
print("Fetching 7 days of historical data...")

historical_start = datetime.now() - timedelta(days=7)
historical_end = datetime.now()

params_historical = {
    'from': historical_start.strftime('%Y-%m-%dT%H:%M:%SZ'),
    'to': historical_end.strftime('%Y-%m-%dT%H:%M:%SZ')
}

response_hist = requests.get(elexon_url, params=params_historical, timeout=30)

if response_hist.status_code == 200:
    data_hist = response_hist.json()
    print(f"Fetched {len(data_hist['data'])} settlement periods")
    
    # Convert to DataFrame
    records_hist = []
    for period in data_hist['data']:
        timestamp = period['startTime']
        for entry in period['data']:
            records_hist.append({
                'timestamp': pd.to_datetime(timestamp),
                'fuel_type': entry['psrType'],
                'generation_mw': entry['quantity']
            })
    
    df_hist = pd.DataFrame(records_hist).sort_values('timestamp').reset_index(drop=True)
    
    print(f"📊 Historical data shape: {df_hist.shape}")
    print(f"Date range: {df_hist['timestamp'].min()} to {df_hist['timestamp'].max()}")
    print(f"Total records: {len(df_hist):,}")
else:
    print(f"Error: {response_hist.status_code}")

Fetching 7 days of historical data...
Fetched 332 settlement periods
📊 Historical data shape: (3652, 3)
Date range: 2026-01-09 12:30:00+00:00 to 2026-01-16 10:30:00+00:00
Total records: 3,652


In [12]:
# Calculate daily renewable percentage
df_hist_pivot = df_hist.pivot_table(
    index='timestamp', 
    columns='fuel_type', 
    values='generation_mw',
    aggfunc='sum'
).fillna(0)

# Define renewables
renewables = ['Solar', 'Wind Offshore', 'Wind Onshore', 
              'Hydro Run-of-river and poundage', 'Hydro Pumped Storage']

df_hist_pivot['total'] = df_hist_pivot.sum(axis=1)
df_hist_pivot['renewables'] = df_hist_pivot[renewables].sum(axis=1)
df_hist_pivot['renewable_pct'] = (df_hist_pivot['renewables'] / df_hist_pivot['total']) * 100

print("📈 Renewable percentage statistics (last 7 days):")
print(f"  Mean: {df_hist_pivot['renewable_pct'].mean():.1f}%")
print(f"  Min: {df_hist_pivot['renewable_pct'].min():.1f}%")
print(f"  Max: {df_hist_pivot['renewable_pct'].max():.1f}%")
print(f"  Std Dev: {df_hist_pivot['renewable_pct'].std():.1f}%")



📈 Renewable percentage statistics (last 7 days):
  Mean: 43.4%
  Min: 20.8%
  Max: 100.0%
  Std Dev: 13.2%


In [13]:
# Plot renewable percentage over time
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_hist_pivot.index,
    y=df_hist_pivot['renewable_pct'],
    mode='lines',
    name='Renewable %',
    line=dict(color='#6BCB77', width=2),
    fill='tozeroy',
    fillcolor='rgba(107, 203, 119, 0.3)'
))

fig.add_hline(y=df_hist_pivot['renewable_pct'].mean(), 
              line_dash="dash", 
              line_color="red",
              annotation_text=f"Mean: {df_hist_pivot['renewable_pct'].mean():.1f}%")

fig.update_layout(
    title='UK Renewable Energy Percentage - Last 7 Days',
    xaxis_title='Date',
    yaxis_title='Renewable Percentage (%)',
    height=500,
    hovermode='x unified'
)

fig.show()

In [14]:
# Find the 100% renewable moment(s)
high_renewable = df_hist_pivot[df_hist_pivot['renewable_pct'] > 95].copy()

print(f"🌟 Periods with >95% renewable generation: {len(high_renewable)}")
print("\nTop 10 highest renewable periods:")
top_renewable = df_hist_pivot.nlargest(10, 'renewable_pct')[['total', 'renewables', 'renewable_pct']]
print(top_renewable)

# Show the breakdown at 100% renewable moment
if len(high_renewable) > 0:
    max_renewable_time = df_hist_pivot['renewable_pct'].idxmax()
    print(f"\n⚡ Peak renewable moment: {max_renewable_time}")
    
    # Get the generation breakdown at that moment
    peak_data = df_hist[df_hist['timestamp'] == max_renewable_time]
    peak_summary = peak_data.groupby('fuel_type')['generation_mw'].sum().sort_values(ascending=False)
    
    print("\n💚 Generation breakdown at peak renewable:")
    for fuel, mw in peak_summary.items():
        print(f"  {fuel:30s}: {mw:8,.0f} MW")

🌟 Periods with >95% renewable generation: 4

Top 10 highest renewable periods:
fuel_type                      total  renewables  renewable_pct
timestamp                                                      
2026-01-15 10:00:00+00:00  14513.210   14513.210     100.000000
2026-01-15 10:30:00+00:00  14116.809   14116.809     100.000000
2026-01-15 11:00:00+00:00  13012.536   13012.536     100.000000
2026-01-15 11:30:00+00:00  12714.696   12714.696     100.000000
2026-01-15 03:30:00+00:00  24883.493   16995.493      68.300270
2026-01-15 02:00:00+00:00  25578.848   17451.848      68.227654
2026-01-15 02:30:00+00:00  25284.333   17166.333      67.893161
2026-01-15 03:00:00+00:00  24767.939   16783.939      67.764778
2026-01-15 04:00:00+00:00  24460.583   16525.583      67.560054
2026-01-15 04:30:00+00:00  24259.605   16301.605      67.196498

⚡ Peak renewable moment: 2026-01-15 10:00:00+00:00

💚 Generation breakdown at peak renewable:
  Wind Offshore                 :    9,330 MW
  Wind Onsho

In [17]:
# Create a detailed view around the 100% moment
if len(high_renewable) > 0:
    # Get 6 hours around the peak
    peak_time = df_hist_pivot['renewable_pct'].idxmax()
    window_start = peak_time - timedelta(hours=6)
    window_end = peak_time + timedelta(hours=6)
    
    window_data = df_hist_pivot.loc[window_start:window_end]
    
    # Create detailed breakdown chart
    fig = go.Figure()
    
    # Add each fuel type
    fuel_colors = {
        'Nuclear': '#FF6B6B',
        'Fossil Gas': '#8B4513', 
        'Wind Offshore': '#4ECDC4',
        'Wind Onshore': '#95E1D3',
        'Solar': '#FFD93D',
        'Biomass': '#6BCB77',
        'Hydro Run-of-river and poundage': '#6C5CE7',
        'Hydro Pumped Storage': '#4D96FF',
        'Other': '#95A5A6'
    }
    
    for fuel_type in window_data.columns:
        if fuel_type not in ['total', 'renewables', 'renewable_pct']:
            fig.add_trace(go.Scatter(
                x=window_data.index,
                y=window_data[fuel_type],
                name=fuel_type,
                mode='lines',
                stackgroup='one',
                fillcolor=fuel_colors.get(fuel_type, '#95A5A6'),
                line=dict(width=0.5)
            ))
    
    # Add vertical line at peak using add_shape instead
    fig.add_shape(
        type="line",
        x0=peak_time, x1=peak_time,
        y0=0, y1=1,
        yref='paper',
        line=dict(color="green", width=2, dash="dash")
    )
    
    # Add annotation separately
    fig.add_annotation(
        x=peak_time,
        y=1,
        yref='paper',
        text="100% Renewable!",
        showarrow=True,
        arrowhead=2,
        arrowcolor="green",
        bgcolor="lightgreen",
        bordercolor="green"
    )
    
    fig.update_layout(
        title=f'Path to 100% Renewable - {peak_time.strftime("%Y-%m-%d %H:%M")}',
        xaxis_title='Time',
        yaxis_title='Generation (MW)',
        height=600,
        hovermode='x unified'
    )
    
    fig.show()

In [18]:
# Investigate the 100% renewable periods more carefully
print("🔍 Investigating the 100% renewable claim...")

# Compare total generation at "100%" vs normal periods
normal_periods = df_hist_pivot[df_hist_pivot['renewable_pct'] < 50]
peak_periods = df_hist_pivot[df_hist_pivot['renewable_pct'] > 95]

print(f"\n📊 Total generation comparison:")
print(f"  Normal periods average: {normal_periods['total'].mean():,.0f} MW")
print(f"  Peak renewable average: {peak_periods['total'].mean():,.0f} MW")
print(f"  Difference: {normal_periods['total'].mean() - peak_periods['total'].mean():,.0f} MW")

print(f"\n⚠️ Suspiciously low total at '100%' renewable!")
print(f"  UK typical demand: 30,000-40,000 MW")
print(f"  These periods show only: {peak_periods['total'].mean():,.0f} MW")

# Check what's happening with nuclear
print(f"\n☢️ Nuclear generation at '100%' periods:")
for timestamp in peak_periods.index:
    nuclear_data = df_hist[(df_hist['timestamp'] == timestamp) & (df_hist['fuel_type'] == 'Nuclear')]
    if not nuclear_data.empty:
        print(f"  {timestamp}: {nuclear_data['generation_mw'].values[0]} MW")
    else:
        print(f"  {timestamp}: NO DATA")

# Check if any fuel types are missing data
print(f"\n🔍 Checking for missing fuel types at peak periods:")
for timestamp in peak_periods.index[:2]:  # Check first two
    period_data = df_hist[df_hist['timestamp'] == timestamp]
    fuel_types_present = set(period_data['fuel_type'].unique())
    print(f"\n{timestamp}:")
    print(f"  Fuel types present: {len(fuel_types_present)}")
    print(f"  Missing: {set(renewables + ['Nuclear', 'Fossil Gas', 'Fossil Hard coal']) - fuel_types_present}")

🔍 Investigating the 100% renewable claim...

📊 Total generation comparison:
  Normal periods average: 34,845 MW
  Peak renewable average: 13,589 MW
  Difference: 21,255 MW

⚠️ Suspiciously low total at '100%' renewable!
  UK typical demand: 30,000-40,000 MW
  These periods show only: 13,589 MW

☢️ Nuclear generation at '100%' periods:
  2026-01-15 10:00:00+00:00: 0.0 MW
  2026-01-15 10:30:00+00:00: 0.0 MW
  2026-01-15 11:00:00+00:00: 0.0 MW
  2026-01-15 11:30:00+00:00: 0.0 MW

🔍 Checking for missing fuel types at peak periods:

2026-01-15 10:00:00+00:00:
  Fuel types present: 11
  Missing: set()

2026-01-15 10:30:00+00:00:
  Fuel types present: 11
  Missing: set()


In [19]:
# Deep dive into the suspicious periods
print("🚨 DATA QUALITY ISSUE ")


suspicious_time = pd.Timestamp('2026-01-15 10:00:00+00:00')

# Check data around that time
window = df_hist[(df_hist['timestamp'] >= suspicious_time - timedelta(hours=1)) & 
                  (df_hist['timestamp'] <= suspicious_time + timedelta(hours=1))]

# Pivot to see all fuel types
window_pivot = window.pivot(index='timestamp', columns='fuel_type', values='generation_mw')

print("\nGeneration data around suspicious '100%' period:")
print(window_pivot[['Nuclear', 'Fossil Gas', 'Wind Offshore', 'Wind Onshore', 'Solar']])

print("\nAnalysis:")
print("  - Nuclear dropped to 0 (IMPOSSIBLE - nuclear runs 24/7)")
print("  - Fossil Gas dropped to 0 (unlikely)")
print("  - Total generation = only renewables reporting")
print("  - This is INCOMPLETE DATA from the API")

print("\nReal renewable % (when all data present):")
complete_data = df_hist_pivot[df_hist_pivot['total'] > 25000]  # Filter for complete data
print(f"  Mean: {complete_data['renewable_pct'].mean():.1f}%")
print(f"  Max: {complete_data['renewable_pct'].max():.1f}%")
print(f"  This is the ACTUAL story!")

🚨 DATA QUALITY ISSUE 

Generation data around suspicious '100%' period:
fuel_type                  Nuclear  Fossil Gas  Wind Offshore  Wind Onshore  \
timestamp                                                                     
2026-01-15 09:00:00+00:00   3900.0     16240.0       9486.280      4567.966   
2026-01-15 09:30:00+00:00   3903.0     16374.0       9543.795      4470.384   
2026-01-15 10:00:00+00:00      0.0         0.0       9330.057      4173.153   
2026-01-15 10:30:00+00:00      0.0         0.0       8711.061      4053.748   
2026-01-15 11:00:00+00:00      0.0         0.0       7838.509      3761.027   

fuel_type                   Solar  
timestamp                          
2026-01-15 09:00:00+00:00   328.0  
2026-01-15 09:30:00+00:00   651.0  
2026-01-15 10:00:00+00:00  1010.0  
2026-01-15 10:30:00+00:00  1352.0  
2026-01-15 11:00:00+00:00  1413.0  

Analysis:
  - Nuclear dropped to 0 (IMPOSSIBLE - nuclear runs 24/7)
  - Fossil Gas dropped to 0 (unlikely)
  - Total gene

In [21]:
# Create cleaned dataset
print("🧹 Cleaning data...")

# Filter for periods with realistic total generation (>25 GW)
df_hist_clean = df_hist_pivot[df_hist_pivot['total'] > 25000].copy()

print(f"Removed {len(df_hist_pivot) - len(df_hist_clean)} suspicious records")
print(f"Remaining: {len(df_hist_clean)} valid periods")

print("\n📊 Real UK Renewable Statistics (cleaned data):")
print(f"  Mean: {df_hist_clean['renewable_pct'].mean():.1f}%")
print(f"  Max: {df_hist_clean['renewable_pct'].max():.1f}%")
print(f"  Min: {df_hist_clean['renewable_pct'].min():.1f}%")
print(f"  Std Dev: {df_hist_clean['renewable_pct'].std():.1f}%")

# Find the REAL peak
real_peak_time = df_hist_clean['renewable_pct'].idxmax()
real_peak_value = df_hist_clean['renewable_pct'].max()

print(f"\n🌟 Real peak renewable: {real_peak_value:.1f}% at {real_peak_time}")

# Show breakdown at real peak
real_peak_data = df_hist[df_hist['timestamp'] == real_peak_time]
print("\n💚 Generation at real peak:")
for fuel, mw in real_peak_data.groupby('fuel_type')['generation_mw'].sum().sort_values(ascending=False).items():
    if mw > 0:
        print(f"  {fuel:30s}: {mw:8,.0f} MW")

🧹 Cleaning data...
Removed 58 suspicious records
Remaining: 274 valid periods

📊 Real UK Renewable Statistics (cleaned data):
  Mean: 41.8%
  Max: 68.2%
  Min: 20.8%
  Std Dev: 11.7%

🌟 Real peak renewable: 68.2% at 2026-01-15 02:00:00+00:00

💚 Generation at real peak:
  Wind Offshore                 :   12,035 MW
  Wind Onshore                  :    4,981 MW
  Nuclear                       :    3,913 MW
  Fossil Gas                    :    2,595 MW
  Biomass                       :    1,354 MW
  Hydro Run-of-river and poundage:      434 MW
  Other                         :      265 MW
  Hydro Pumped Storage          :        2 MW


In [22]:
# Import our new production code
import sys
sys.path.insert(0, '../')

from src.data.fetch_data import ElexonDataFetcher

# Test it
print("Testing our production fetcher...")
fetcher = ElexonDataFetcher()

# Fetch current data
current_data = fetcher.fetch_current_generation()
print(f"\n Fetched {len(current_data)} records")
print(current_data.head())

# Save to CSV
fetcher.save_to_csv(current_data, 'production_test.csv')
print("\n Production code works!")

INFO:src.data.fetch_data:Fetching generation data from 2026-01-15 13:16:33.663660 to 2026-01-16 13:16:33.663660


Testing our production fetcher...


INFO:src.data.fetch_data:API request completed in 222.02ms - Status: 200
INFO:src.data.fetch_data:Successfully fetched 495 records
INFO:src.data.fetch_data:Data saved to c:\Users\dhruv\uk-energy-grid\notebooks\..\data\raw\production_test.csv



 Fetched 330 records
                  timestamp             fuel_type  generation_mw  \
0 2026-01-15 13:30:00+00:00               Biomass         1387.0   
1 2026-01-15 13:30:00+00:00            Fossil Gas        19964.0   
2 2026-01-15 13:30:00+00:00      Fossil Hard coal            0.0   
3 2026-01-15 13:30:00+00:00            Fossil Oil            0.0   
4 2026-01-15 13:30:00+00:00  Hydro Pumped Storage           76.0   

   total_generation  
0         37171.456  
1         37171.456  
2         37171.456  
3         37171.456  
4         37171.456  

 Production code works!
